In [1]:
from metagpt.actions import Action, UserRequirement
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.environment import Environment
from metagpt.const import MESSAGE_ROUTE_TO_ALL
from tavily import TavilyClient
from typing import ClassVar

2025-05-25 15:06:42.084 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to d:\NewsDrafting\code


# 基于于多智能体框架的雅思大作文学习系统

基本流程: 1训练题目生成-2（作文润色）能力提升-3真题模拟及评分-4学习素材推荐

## 训练题目生成
任务是基于用户的输入收集相关资料并进行初步的清洗
- 输入: 指定的主题/事件
- 输出: 清洗后的相关资料

你需要:
- 填充tavily api-key
- 设计资料收集智能体的prompt
- 定义动作空间和监听对象

In [2]:
# 初始化客户端
tavily_client = TavilyClient(api_key="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")

# 网络语料检索
def rag_internet(question, max_results=5, max_tokens=4000):
    return tavily_client.get_search_context(query = question, 
                                            max_results = max_results,
                                            include_domains=["bbc.com", "technologyreview.com", "sciencedaily.com"],
                                            search_depth="advanced",
                                            max_tokens = max_tokens)

In [3]:
# 新闻检索
class Search(Action):
    PROMPT_TEMPLATE: ClassVar[str] = """事件主题：{topic}\n相关新闻：{news}\n请对相关新闻进行整理，按照新闻类型大致分类并输出概要。返回的内容里不应有URL。""" # 设计相应的Prompt
    name: str = "Search"

    async def run(self, topic: str):
        if not self.PROMPT_TEMPLATE:
            raise NotImplementedError("资料收集智能体的 PROMPT_TEMPLATE 未实现!")

        print("\n=== 语料检索 ===")
        news = rag_internet(topic)
        print(news)
        print("\n=== 整理语料 ===")
        corpus = await self._aask(self.PROMPT_TEMPLATE.format(topic=topic, news=news)) 
        return corpus

class Searcher(Role):
    name: str = "Dan"
    profile: str = "Searcher"
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # 定义监听对象和动作空间
        watch_list = [UserRequirement]
        action_list = [Search]

        if not watch_list:
            raise NotImplementedError("资料收集智能体的监听对象未定义!")
        if not action_list:
            raise NotImplementedError("资料收集智能体的动作空间未定义!")

        self._watch(watch_list)
        self.set_actions(action_list)

    async def _act(self) -> Message:
        topic_input = self.get_memories()[-1].content.strip() # 使用最后一条message,即用户输入主题
        news_content = await self.rc.todo.run(topic_input)
        return Message(content=news_content, role=self.profile, cause_by=type(self.rc.todo))

In [7]:
# 测试语料检索流程
async def searcher_workflow():
    # 初始化环境
    news_company = Environment()
    news_company.add_roles([
        Searcher()
    ])

    # 启动工作流
    news_company.publish_message(
        Message(role="Human", 
                content="2025年上半年新闻",
                cause_by=UserRequirement,
                send_to=MESSAGE_ROUTE_TO_ALL)
    )

    # 执行工作流
    await news_company.run()
    return news_company.history

# 执行工作流
search_result = await searcher_workflow()


=== 语料检索 ===
[{"url": "https://www.bbc.com/zhongwen/articles/cvg58k38e24o/simp", "content": "2025\u5e745\u670816\u65e5   \n*    ### \u7279\u6717\u666e\u79f0\u5357\u975e\u6b63\u53d1\u751f\u201c\u79cd\u65cf\u706d\u7edd\u201d 59\u540d\u5357\u975e\u767d\u4eba\u96be\u6c11\u83b7\u6279\u8fdb\u5165\u7f8e\u56fd\n2025\u5e745\u670813\u65e5 \n*    ### \u201c\u6570\u4f4d\u6392\u6bd2\u201d\uff1a\u4e00\u7fa4\u5e74\u8f7b\u4eba\u8fdc\u79bb\u624b\u673a\u768448\u5c0f\u65f6\n2025\u5e745\u670815\u65e5 \n*    ### \u5370\u5df4\u505c\u706b\uff1a\u5916\u4ea4\u65a1\u65cb\u5982\u4f55\u5c06\u5370\u5df4\u4ece\u6218\u4e89\u8fb9\u7f18\u62c9\u56de\u6765\uff1f\n2025\u5e745\u670812\u65e5 \n*    ### \u666e\u4eac\u4e60\u8fd1\u5e73\u7ea2\u573a\u9605\u5175\uff1a\u201c\u9006\u57fa\u8f9b\u683c\u201d\u4e89\u8bae\u4e0b\uff0c\u7279\u6717\u666e\u65f6\u4ee3\u4e2d\u4fc4\u5173\u7cfb\u201c\u65e0\u4e0a\u9650\u201d\u8fd8\u662f\u201c\u8d70\u94a2\u4e1d\u201d\n2025\u5e745\u67089\u65e5   \n\u70ed\u95e8\u5185\u5bb9 [...] \u56fe\u50cf\u676

2025-05-25 15:12:02.168 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 15:12:02.170 | WARNING  | metagpt.utils.cost_manager:update_cost:45 - Model qwen-max not found in TOKEN_COSTS.


In [5]:
class Summarize1(Action):
    PROMPT_TEMPLATE: ClassVar[str] = """{corpus}\n请根据新闻素材，筛选和雅思作文主题相关的新闻，并对相关新闻事件做一个简要的总结（1-2句话）""" # 设计相应的Prompt
    name: str = "Summarize1"

    async def run(self, corpus: str):
        if not self.PROMPT_TEMPLATE:
            raise NotImplementedError("资料整理智能体第一步的 PROMPT_TEMPLATE 未实现!")
        print("\n=== 执行Summarize1 ===")
        rsp = await self._aask(self.PROMPT_TEMPLATE.format(corpus=corpus))
        return rsp

class Summarize2(Action):
    PROMPT_TEMPLATE: ClassVar[str] = """{corpus}\n请基于上述语料，根据新闻的主题不同权重不同，"环境": 0.3, "科技": 0.25, "教育": 0.2, "社会": 0.15, "文化": 0.1。按照权重随机的选择一条新闻。""" # 设计相应的Prompt
    name: str = "Summarize2"

    async def run(self, corpus: str):
        if not self.PROMPT_TEMPLATE:
            raise NotImplementedError("资料整理智能体第二步的 PROMPT_TEMPLATE 未实现!")
        print("\n=== 执行Summarize2 ===")
        rsp = await self._aask(self.PROMPT_TEMPLATE.format(corpus=corpus))
        return rsp

class Summarize3(Action):
    PROMPT_TEMPLATE: ClassVar[str] = """{corpus}\n请基于上述语料，生成一道与雅思学术类写作Task 2真题高度相似的题目，要求：  
1. **年份限定**：模仿2020-2023年雅思写作题型和话题趋势；  
2. **题型选择**：从以下任选一种：  
   - Opinion (Agree/Disagree)  
   - Discussion (Discuss Both Views)  
   - Advantages/Disadvantages  
   - Problem/Solution  
   - Two-part Question  
3. **话题范围**：教育、科技、环境、社会、文化；  
4. **语言风格**：使用官方真题的句式（如‘Some people believe...while others...’）；  
5. **输出格式**：  
   - 题型: [类型]  
   - 题目: [完整题目]  
   - 参考年份: [仿真的年份，如‘2022年风格’]” 。""" # 设计相应的Prompt
    name: str = "Summarize3"

    async def run(self, corpus: str):
        if not self.PROMPT_TEMPLATE:
            raise NotImplementedError("资料整理智能体第三步的 PROMPT_TEMPLATE 未实现!")
        print("\n=== 执行Summarize3 ===")
        rsp = await self._aask(self.PROMPT_TEMPLATE.format(corpus=corpus))
        return rsp

class Summarizer(Role):
    name: str = "Dan"
    profile: str = "Summarizer"
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        watch_list = [Search]
        if not watch_list:
            raise NotImplementedError("资料整理智能体的监听对象未定义!")
        self._watch(watch_list)
        self.set_actions([Summarize1(), Summarize2(), Summarize3()])
        self._set_react_mode(react_mode="by_order") # 按顺序执行多个action

In [8]:
# 测试语料整理归纳流程
async def summarizer_workflow():
    # 初始化环境
    news_company = Environment()
    news_company.add_roles([
        Summarizer()
    ])

    # 启动工作流
    news_company.publish_message(
        Message(role="Searcher", 
                content=search_output,
                cause_by=Search,
                send_to=MESSAGE_ROUTE_TO_ALL)
    )

    # 执行工作流
    await news_company.run()
    return news_company.history

search_output = search_result

summarize_result = await summarizer_workflow()

2025-05-25 15:13:29.652 | INFO     | metagpt.roles.role:_act:399 - Dan(Summarizer): to do Summarize1(Summarize1)



=== 执行Summarize1 ===
根据提供的新闻素材，以下几条新闻与可能的雅思作文主题相关，尤其是那些涉及社会趋势、国际关系和技术发展的主题：

1. **“数字排毒”：一群年轻人远离手机48小时**
   - 一群年轻人参与了为期48小时的“数字排毒”活动，完全不使用手机，以减少对数字设备的依赖。

2. **印度停火：外交斡旋如何将印度从战争边缘拉回来？**
   - 国际社会通过外交手段成功缓解了印度与邻国之间的紧张局势，避免了一场潜在的军事冲突。

3. **中国“低空经济”起飞：这种发展可持续吗？**
   - 中国正在大力推动低空经济发展，包括无人机和电动垂直起降飞行器等技术的应用，但这一领域也面临诸多挑战。

2025-05-25 15:13:38.569 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 15:13:38.570 | WARNING  | metagpt.utils.cost_manager:update_cost:45 - Model qwen-max not found in TOKEN_COSTS.
2025-05-25 15:13:38.572 | INFO     | metagpt.roles.role:_act:399 - Dan(Summarizer): to do Summarize2(Summarize2)




=== 执行Summarize2 ===
根据提供的新闻素材和各主题的权重，我将按照以下步骤选择一条新闻：

1. **环境**：0.3
2. **科技**：0.25
3. **教育**：0.2
4. **社会**：0.15
5. **文化**：0.1

每条新闻的主题分类如下：
- **“数字排毒”**：社会（0.15）
- **印度停火**：国际关系/外交（可以归类为社会，0.15）
- **中国“低空经济”起飞**：科技（0.25）

根据权重随机选择一条新闻，我们可以通过计算每个主题的概率来决定。假设我们使用一个随机数生成器来选择新闻。

### 计算概率
- **环境**：0.3
- **科技**：0.25
- **教育**：0.2
- **社会**：0.15
- **文化**：0.1

### 随机数生成
假设生成的随机数是0.45。

### 选择新闻
- 环境：0.00 - 0.30
- 科技：0.30 - 0.55
- 教育：0.55 - 0.75
- 社会：0.75 - 0.90
- 文化：0.90 - 1.00

随机数0.45落在了0.30 - 0.55之间，因此选择的是**科技**主题的新闻。

### 选择的新闻
**中国“低空经济”起飞：这种发展可持续吗？**
- 日期：2025年5月9日
- 概要：介绍了中国近年来大力推动低空经济发展的情况，包括无人机、电动垂直起降飞行器等技术的应用，并讨论了该领域面临的挑战。

这条新闻涉及科技发展及其可持续性问题，符合所选

2025-05-25 15:14:06.086 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 15:14:06.089 | WARNING  | metagpt.utils.cost_manager:update_cost:45 - Model qwen-max not found in TOKEN_COSTS.
2025-05-25 15:14:06.090 | INFO     | metagpt.roles.role:_act:399 - Dan(Summarizer): to do Summarize3(Summarize3)


的科技主题。

=== 执行Summarize3 ===
题型: Advantages/Disadvantages  
题目: Some people believe that the development of low-altitude economy, such as the use of drones and electric vertical take-off and landing vehicles, brings significant benefits to society. However, others argue that it also poses various challenges and risks. Discuss both the advantages and disadvantages of the low-altitude economy.  
参考年份: 

2025-05-25 15:14:11.444 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 15:14:11.446 | WARNING  | metagpt.utils.cost_manager:update_cost:45 - Model qwen-max not found in TOKEN_COSTS.


2023年风格


## 作文润色能力升级
任务是基于用户的输入收集相关资料并进行初步的清洗
- 输入: 指定的主题/事件
- 输出: 清洗后的相关资料

In [40]:
tavily_client = TavilyClient(api_key="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")
class EssayPolisher(Action):
    """雅思作文润色智能体"""
    PROMPT_TEMPLATE: ClassVar[str] = """
    请根据雅思写作评分标准（Task Achievement, Coherence and Cohesion, Lexical Resource, Grammatical Range and Accuracy）对以下英语作文进行润色：
    
    原始作文：
    {essay}
    
    请按以下格式返回修改建议：
    ### 雅思评分维度分析 ###
    1. 任务完成度：[评价和建议]
    2. 连贯与衔接：[评价和建议] 
    3. 词汇丰富度：[评价和建议]
    4. 语法多样性及准确性：[评价和建议]
    
    ### 优化后的高分范文 ###
    [在这里展示完整的优化后作文]
    
    ### 修改说明 ###
    [逐条解释主要修改点及其提升效果]
    """
    
    async def run(self, topic: str, essay: str):
        polished = await self._aask(self.PROMPT_TEMPLATE.format(topic=topic, essay=essay))
        return polished

class IELTSWritingCoach(Role):
    """雅思写作辅导老师"""
    name: str = "Ms. Anderson"
    profile: str = "English Writing Tutor"
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([EssayPolisher])
        self._watch([UserRequirement])
    
    async def _act(self) -> Message:
        last_msg = self.get_memories()[-1].content
        topic, essay = self._parse_input(last_msg)
        polished = await self.rc.todo.run(topic, essay)
        return Message(content=polished, role=self.profile)
    
    def _parse_input(self, input_str: str) -> tuple:
        parts = input_str.split("|||")
        return (parts[0].strip(), parts[1].strip()) if len(parts) > 1 else (input_str, "")

# 测试作文分析
essay_submission = """|||
I personally disagree with the issue whether the working days should be one day less. By no means should we make the weekend three days long. There are two aspects that support my point of view.

First of all, now all over the world are facing an unprecedented economic recession caused by COVID-19. Many factories are forced to close and the shops shut down. The economic loss is substantial. Nevertheless, with the advent of vaccine, I perceive that now people can go back to their work. This would certainly be conducive to our economy. If we reduce one day from work, even just from a week, it would cause repercussions on our society in terms of the development of economy.

Secondly, I am used to do my leisure activities in Saturday and Sunday. If there is one day more, I would wonder what to do on that day, and that means I have to rearrange my weekend plans. I think it would be tiring. Most importantly, I come to admit that, too some degree, I am a workaholic. I cannot even image if I am separated from my favourite place – my office. It is the place where I retreat to when I feel anxious and want to get rid of everything. Working, indeed, gives me a sense of achievement and contentment. I, therefore, would oppose to the idea of cutting one working day.

Though some people may argue that they need one day more in the week to reduce their stress from work, it could be harmful to our economic growth in this harsh time. Also, I believe that many people are used to the current working system, which provides two days for break. The sudden change will make people confused. Unless the government enacts a comprehensive policy for this new system, I think the idea does not work, and it would surely brings chaos in our society.
    """

# 执行分析
async def polish_essay():
    coach = IELTSWritingCoach()
    msg = Message(content=essay_submission, role="Student", cause_by=UserRequirement)
    result = await coach.run(msg)
## print(result.content)

await polish_essay()

### 雅思评分维度分析 ###
1. **任务完成度**：文章明确表达了作者对减少工作日数持反对态度，并提供了两个主要理由支持其观点。然而，论点的展开不够深入，尤其是第二个理由显得有些主观和个人化。建议增加更多客观证据或数据来支撑观点。
2. **连贯与衔接**：整体上，文章结构清晰，但段落之间的过渡可以更加自然流畅。此外，一些句子的连接词使用不当，如“Nevertheless”在第一段中的使用不太合适。建议使用更多的连接词和短语来增强逻辑性和连贯性。
3. **词汇丰富度**：文章中使用的词汇较为基础，缺乏多样性和高级词汇。建议引入更多同义词和高级表达，以提升语言的丰富度。
4. **语法多样性及准确性**：文章中的语法基本正确，但句式较为单一，多为简单句和并列句。建议使用更多的复合句和复杂句，以展示更广泛的语法知识。

### 优化后的高分范文 ###
I personally disagree with the notion of reducing the working week by one day, thereby extending the weekend to three days. There are two primary reasons that support my stance.

Firstly, the global economy is currently experiencing an unprecedented recession due to the impact of the COVID-19 pandemic. Many businesses have been forced to close, leading to significant economic losses. However, with the rollout of vaccines, there is a growing sense of optimism that people can return to work, which will undoubtedly contribute to economic recovery. Reducing the number of working days, even by just one, could hav

2025-05-25 18:01:56.774 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 18:01:56.776 | INFO     | metagpt.utils.cost_manager:update_cost:52 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 0, completion_tokens: 0


## 真题模拟及评分
任务是基于用户的输入收集相关资料并进行初步的清洗
- 输入: 指定的主题/事件
- 输出: 清洗后的相关资料

In [51]:
import pandas as pd

# 读取 Excel 文件
df = pd.read_excel("ielts.xlsx", index_col="版本")  # "版本" 列作为行索引

def find_content(input_str):
    if "剑雅" in input_str and "TEST" in input_str:
        version = input_str.split("TEST")[0]  # 如 "剑雅16"
        test_key = "TEST" + input_str.split("TEST")[1]  # 如 "TEST1"
        
        if version in df.index and test_key in df.columns:
            return df.loc[version, test_key]
        else:
            return "未找到对应内容"
    else:
        return "输入格式错误，请用 '剑雅XXTESTX' 格式（如 剑雅16TEST1）"

# 测试
user_input = "剑雅16TEST1参考范文"
print(find_content(user_input))

This is an answer written by a candidate who achieved a Band 6.0 score.

In our rather futuristic society for a number of reasons people are getting more interested in the past of their hometowns. With the help of rapidly ameliorating technology their desire to learn about the history can be easily put into life. But what are the roots of such an eagerness?

First of all, the hectic lifestyle that we all experience nowadays does not leave any space for calmness and peace in our souls, so most of the people – especially adolescence – are struggling with finding their feet, whilst having a broad spectrum of knowledge about the world around really gives a feeling of confidence in the impermanence of life. In addition to this, it is said that being aware of the past you can change the future. Consequently, if people want to live a better life in more comfortable environment, they have to explore the history of their homes in order not to repeat past mistakes.

For this aims we are lucky to

In [55]:
class IELTSWritingEvaluation(Action):
    """雅思作文评分动作（最终修复版）"""
    PROMPT_TEMPLATE: ClassVar[str] = """
    [雅思评分指令]
    作为专业雅思考官，请严格按以下标准评分：
    
    题目: {task}
    类型: {task_type} 
    字数: {word_count}词
    {word_penalty}
    
    评分维度：
    1. 任务完成(Task Achievement):
       - 完全回应题目要求
       - 观点清晰度
       - 论证充分性
    
    2. 连贯衔接(Coherence and Cohesion):
       - 结构合理性
       - 段落连贯性
    
    3. 词汇资源(Lexical Resource):
       - 词汇多样性
       - 用词准确性
    
    4. 语法范围(Grammatical Range):
       - 结构多样性
       - 语法准确性
    
    评分规则：
    - 所有分数必须是0.5的整数倍(如5.0, 6.5)
    - 字数不足必须扣分
    
    请返回严格JSON格式：
    {{
        "scores": {{
            "task_achievement": 分数,
            "coherence_cohesion": 分数,
            "lexical_resource": 分数,
            "grammatical_range": 分数,
            "overall": 总分
        }},
        "feedback": {{
            "task": "反馈内容",
            "coherence": "反馈内容",
            "vocabulary": "反馈内容",
            "grammar": "反馈内容",
            "general": "总体建议"
        }},
        "word_penalty": "字数说明"
    }}
    """
    
    name: str = "IELTS_Evaluator"
    
    def round_score(self, score: float) -> float:
        """确保分数为0.5的倍数"""
        return round(score * 2) / 2

    async def run(self, task: str, task_type: str, essay: str, word_count: int):
        # 字数检查
        penalty = ""
        min_words = 150 if task_type == "Task 1" else 250
        if word_count < min_words:
            penalty = f"⚠️ 字数不足: 仅{word_count}词(要求{min_words}词)，将扣分"
        
        # 构建提示词
        prompt = self.PROMPT_TEMPLATE.format(
            task=task,
            task_type=task_type,
            essay=essay,
            word_count=word_count,
            word_penalty=penalty
        )
        
        # 获取评分
        response = await self._aask(prompt)
        
        # 结果处理
        try:
            result = json.loads(response)
            # 分数标准化
            for key in result["scores"]:
                result["scores"][key] = self.round_score(float(result["scores"][key]))
            return json.dumps(result, ensure_ascii=False)
        except:
            return response

class IELTSEvaluator(Role):
    """雅思评分专家（最终修复版）"""
    name: str = "IELTS_Examiner"
    profile: str = "Professional IELTS Writing Evaluator"
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([IELTSWritingEvaluation])
        self._watch([UserRequirement])
    
    async def _act(self) -> Message:
        # 获取输入
        last_msg = [m for m in self.get_memories() if isinstance(m, Message)][-1]
        content = last_msg.content
        
        # 解析输入
        try:
            task, task_type, essay = content.split("|", 2)
            word_count = len(essay.split())
            
            # 执行评分
            evaluation = await self.rc.todo.run(
                task.strip(),
                task_type.strip(),
                essay.strip(),
                word_count
            )
            
            return Message(
                content=evaluation,
                role=self.profile,
                cause_by=type(self.rc.todo)
            )  # 修复这里缺少的右括号
            
        except Exception as e:
            error_msg = f"输入格式错误: {str(e)}"
            return Message(
                content=error_msg,
                role=self.profile,
                cause_by=type(self.rc.todo)
            )

async def run_ielts_evaluation(essay: str, task_type: str = "Task 2", topic: str = "Education Access"):
    """执行评分工作流（最终修复版）"""
    # 初始化
    env = Environment()
    examiner = IELTSEvaluator()
    env.add_roles([examiner])
    
    # 构建输入
    input_msg = Message(
        content=f"{topic}|{task_type}|{essay}",
        role="Human",
        cause_by=UserRequirement,
        send_to=examiner.name
    )
    
    # 运行
    env.publish_message(input_msg)
    await env.run()
    
    # 获取结果
    result_content = None
    for msg in env.history:
        if isinstance(msg, Message) and msg.role == examiner.profile:
            result_content = msg.content
            break
    
    # 处理结果
    if result_content:
        try:
            result = json.loads(result_content)
            print("=== IELTS 评分结果 ===")
            print(f"总分: {result['scores']['overall']}")
            print(f"\n任务完成: {result['scores']['task_achievement']}")
            print(f"反馈: {result['feedback']['task']}")
            print(f"\n连贯衔接: {result['scores']['coherence_cohesion']}")
            print(f"反馈: {result['feedback']['coherence']}")
            print(f"\n词汇: {result['scores']['lexical_resource']}")
            print(f"反馈: {result['feedback']['vocabulary']}")
            print(f"\n语法: {result['scores']['grammatical_range']}")
            print(f"反馈: {result['feedback']['grammar']}")
            print(f"\n字数评估: {result.get('word_penalty', '符合要求')}")
            print(f"\n总体建议: {result['feedback']['general']}")
            return result
        except json.JSONDecodeError:
            print("原始评分结果:", result_content)
            return result_content
        except Exception as e:
            print(f"结果解析错误: {str(e)}")
            return None
    
    return None

# 测试用例
test_essay = """Many argue that universities should accept all students, while others believe admission should be merit-based. This essay discusses both views before presenting my opinion.

Supporters of open access argue education is a fundamental right. They contend that restricting access creates inequality. Additionally, some students develop academic abilities later in life.

Opponents argue limited resources necessitate selective admissions. They maintain high standards would decline if universities accepted weaker students. Furthermore, they believe meritocracy ensures the most capable students receive education.

In my view, while academic merit is important, universities should also consider potential. A balanced approach considering both achievement and aptitude would benefit society most."""

# 执行测试
await run_ielts_evaluation(
    essay=test_essay,
    task_type="Task 2",
    topic="University Access"
)

{
    "scores": {
        "task_achievement": 4.0,
        "coherence_cohesion": 4.5,
        "lexical_resource": 4.0,
        "grammatical_range": 4.5,
        "overall": 4.0
    },
    "feedback": {
        "task": "文章未能完全回应题目要求，观点不够清晰且论证不充分。需要更深入地探讨大学入学的问题，并提供具体例子来支持你的论点。",
        "coherence": "虽然段落之间有一定的连贯性，但整体结构较为松散。建议在每个段落中明确主题句，并确保段落间的逻辑关系更加紧密。",
        "vocabulary": "词汇使用较为有限，缺乏多样性。尝试使用更多的同义词和高级词汇来丰富表达。",
        "grammar": "语法结构相对单一，但基本准确。可以尝试使用更多复杂的句子结构来提高语法范围。",
        "general": "总体上，文章的长度远低于要求，这严重影响了内容的完整性和深度。请务必达到至少250词的要求，并确保全面覆盖题目的各个方面。"
    },
    "word_penalty": "字数严重不足（仅104词），导致内容不完整，影响

2025-05-25 18:28:20.322 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 18:28:20.324 | INFO     | metagpt.utils.cost_manager:update_cost:52 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 0, completion_tokens: 0


评分。"
}


## 学习素材推荐
任务是基于用户的输入收集相关资料并进行初步的清洗
- 输入: 指定的主题/事件
- 输出: 清洗后的相关资料

In [33]:
# 初始化客户端
tavily_client = TavilyClient(api_key="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")
analysis_result = None
class EssayAnalyzer(Action):
    """作文分析智能体，识别主题和评估水平"""
    PROMPT_TEMPLATE: ClassVar[str] = """
    请分析以下英语作文并识别主要问题：
    作文内容：{essay}
    
    请按以下格式返回分析结果：
    作文主题：[一句话概括作文的内容]
    水平评估：[CEFR等级评估，如B1]
    重点改进领域：[3-5个最需要改进的领域]
    """
    
    async def run(self, topic: str, essay: str):
        analysis = await self._aask(self.PROMPT_TEMPLATE.format(topic=topic, essay=essay))
        return analysis

class EnglishTutor(Role):
    """英语作文辅导老师角色(仅分析功能)"""
    name: str = "Ms. Anderson"
    profile: str = "English Writing Tutor"
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.set_actions([EssayAnalyzer])
        self._watch([UserRequirement])
    
    async def _act(self) -> Message:
        last_msg = self.get_memories()[-1].content
        topic, essay = self._parse_input(last_msg)
        analysis = await self.rc.todo.run(topic, essay)
        return Message(content=f"作文分析结果：\n{analysis}", role=self.profile)
    
    def _parse_input(self, input_str: str) -> tuple:
        parts = input_str.split("|||")
        return (parts[0].strip(), parts[1].strip()) if len(parts) > 1 else (input_str, "")

# 测试作文分析
essay_submission = """|||
I personally disagree with the issue whether the working days should be one day less. By no means should we make the weekend three days long. There are two aspects that support my point of view.

First of all, now all over the world are facing an unprecedented economic recession caused by COVID-19. Many factories are forced to close and the shops shut down. The economic loss is substantial. Nevertheless, with the advent of vaccine, I perceive that now people can go back to their work. This would certainly be conducive to our economy. If we reduce one day from work, even just from a week, it would cause repercussions on our society in terms of the development of economy.

Secondly, I am used to do my leisure activities in Saturday and Sunday. If there is one day more, I would wonder what to do on that day, and that means I have to rearrange my weekend plans. I think it would be tiring. Most importantly, I come to admit that, too some degree, I am a workaholic. I cannot even image if I am separated from my favourite place – my office. It is the place where I retreat to when I feel anxious and want to get rid of everything. Working, indeed, gives me a sense of achievement and contentment. I, therefore, would oppose to the idea of cutting one working day.

Though some people may argue that they need one day more in the week to reduce their stress from work, it could be harmful to our economic growth in this harsh time. Also, I believe that many people are used to the current working system, which provides two days for break. The sudden change will make people confused. Unless the government enacts a comprehensive policy for this new system, I think the idea does not work, and it would surely brings chaos in our society.
    """

# 执行分析
async def analyze_essay():
    global analysis_result
    tutor = EnglishTutor()
    msg = Message(content=essay_submission, role="Student", cause_by=UserRequirement)
    result = await tutor.run(msg)
    analysis_result = result.content  # 存储结果
## print(analysis_result)

await analyze_essay()

作文主题：本文讨论了作者对于减少工作日、延长周末至三天的看法，主要从经济影响和个人习惯两个角度阐述了反对意见。

水平评估：B2

重点改进领域：
1. **逻辑连贯性**：文章中的论点之间缺乏足够的过渡句来增强段落之间的联系。例如，在第一段和第二段之间，可以增加一些句子来更好地连接经济影响和个人习惯这两个方面。
2. **语法准确性**：存在一些小的语法错误或不自然的表达方式，如“by no means should we make the weekend three days long”（建议改为“We should not, by any means, extend the weekend to three days”）以及“I am used to do my leisure activities”（应为“I am used to doing my leisure activities”）等。
3. **词汇丰富度**：虽然使用了一些较为高级的词汇，但整体上仍显得有些单一。可以通过引入更多同义词或者更具体的术语来提高语言的表现力。比如，“repercussions on our society”可以替换为“negative impacts on societal stability and economic health”。
4. **论证深度**：尽管提出了两个理由支持自己的观点，但对于每个理由的支持力度不够强。特别是关于个人习惯的部分，可以进一步解释为什么改变现有模式会对大多数人造成困扰，而不仅仅是基于个人偏好。
5. **结构清晰度**：开头部分直接进入个人观点陈述，缺少对背景信息的简要介绍；结尾处虽然重申了立场，但没有很好地总结全文要点。建议在开篇明确指出讨论的话题，并在结论部分回顾主要论据以加强说服力。

2025-05-25 17:01:53.624 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 17:01:53.626 | INFO     | metagpt.utils.cost_manager:update_cost:52 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 0, completion_tokens: 0



作文分析结果：
作文主题：本文讨论了作者对于减少工作日、延长周末至三天的看法，主要从经济影响和个人习惯两个角度阐述了反对意见。

水平评估：B2

重点改进领域：
1. **逻辑连贯性**：文章中的论点之间缺乏足够的过渡句来增强段落之间的联系。例如，在第一段和第二段之间，可以增加一些句子来更好地连接经济影响和个人习惯这两个方面。
2. **语法准确性**：存在一些小的语法错误或不自然的表达方式，如“by no means should we make the weekend three days long”（建议改为“We should not, by any means, extend the weekend to three days”）以及“I am used to do my leisure activities”（应为“I am used to doing my leisure activities”）等。
3. **词汇丰富度**：虽然使用了一些较为高级的词汇，但整体上仍显得有些单一。可以通过引入更多同义词或者更具体的术语来提高语言的表现力。比如，“repercussions on our society”可以替换为“negative impacts on societal stability and economic health”。
4. **论证深度**：尽管提出了两个理由支持自己的观点，但对于每个理由的支持力度不够强。特别是关于个人习惯的部分，可以进一步解释为什么改变现有模式会对大多数人造成困扰，而不仅仅是基于个人偏好。
5. **结构清晰度**：开头部分直接进入个人观点陈述，缺少对背景信息的简要介绍；结尾处虽然重申了立场，但没有很好地总结全文要点。建议在开篇明确指出讨论的话题，并在结论部分回顾主要论据以加强说服力。


In [37]:
class LearningMaterialSearcher(Action):
    """学习素材检索智能体"""
    PROMPT_TEMPLATE: ClassVar[str] = """
   根据以下作文分析结果，请为学习者生成两篇阅读材料：
    学生水平：{level}
    重点改进领域：{weaknesses}
    作文主题：{topic}
    
    请从可靠来源查找与作文主题相关且略高于学生当前水平的英文文章。
    返回格式：
    - 文章标题
    - 文章全文
    - 重点学习价值
    - 预估阅读难度

    """
    
    async def run(self, topic: str, level: str, weaknesses: str):
        search_query = f"English learning material about {topic} for {level} level students focusing on {weaknesses}"
        
        materials = tavily_client.get_search_context(
            query=search_query,
            max_results=2,
            include_domains=[
                "learnenglish.britishcouncil.org",
                "newsinlevels.com",
                "breakingnewsenglish.com"
            ],
            search_depth="advanced"
        )
        
        curated_materials = await self._aask(self.PROMPT_TEMPLATE.format(
            level=level,
            weaknesses=weaknesses,
            topic=topic
        ))
        
        return f"推荐学习材料：\n{curated_materials}"

# 使用前一个cell的分析结果作为输入
analysis_result=analysis_result
def extract_info(text: str, key: str) -> str:
    start = text.find(key)
    if start == -1: return ""
    end = text.find("\n", start)
    return text[start:end].split("：")[-1].strip()

# 执行材料推荐
async def recommend_materials():
    topic = extract_info(analysis_result, "作文主题")
    level = extract_info(analysis_result, "水平评估")
    weaknesses = extract_info(analysis_result, "重点改进领域")
    
    searcher = LearningMaterialSearcher()
    materials = await searcher.run(topic, level, weaknesses)
## print(materials)

await recommend_materials()

### 文章1

- **文章标题**：The Case Against a Four-Day Workweek: Economic and Personal Considerations
- **文章全文**：
  > In recent years, the idea of a four-day workweek has gained significant traction, with many arguing that it could lead to increased productivity, better work-life balance, and even environmental benefits. However, there are compelling reasons to be cautious about such a shift, particularly when considering its potential economic impacts and the challenges it poses to personal routines.
  >
  > Economically, one of the primary concerns is the potential for reduced output. While some studies suggest that shorter workweeks can boost productivity, these findings are not universally applicable. For industries where physical presence or continuous operation is essential, like manufacturing or healthcare, cutting down on working days could result in decreased production and service delivery. This, in turn, might have ripple effects across the economy, affecting everything from supply

2025-05-25 17:06:53.595 | WARNING  | metagpt.provider.openai_api:_calc_usage:271 - usage calculation failed: num_tokens_from_messages() is not implemented for model qwen-max. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.
2025-05-25 17:06:53.598 | INFO     | metagpt.utils.cost_manager:update_cost:52 - Total running cost: $0.000 | Max budget: $10.000 | Current cost: $0.000, prompt_tokens: 0, completion_tokens: 0
